In [6]:
import pygame
import numpy as np
from typing import List, Tuple
from enum import Enum

# Initialize Pygame
pygame.init()

# Constants
WIDTH = 800
HEIGHT = 600
FPS = 60
G = 6.67430e-11  # gravitational constant
SCALE = 1e9  # scale factor to convert meters to pixels
TIME_STEP = 3600  # simulation time step in seconds (1 hour)
VELOCITY_SCALE = 1e4  # scale factor for launch velocity

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (128, 128, 128)

class BodySize(Enum):
    SMALL = {"mass": 1e23, "radius": 5}
    MEDIUM = {"mass": 1e24, "radius": 8}
    LARGE = {"mass": 1e25, "radius": 12}

class Body:
    def __init__(self, mass: float, position: np.ndarray, velocity: np.ndarray, color: Tuple[int, int, int], radius: int):
        self.mass = mass
        self.position = position  # in meters
        self.velocity = velocity  # in m/s
        self.color = color
        self.radius = radius  # display radius in pixels
        
    def update_position(self, acceleration: np.ndarray, time_step: float):
        self.velocity += acceleration * time_step
        self.position += self.velocity * time_step
        
    def draw(self, screen):
        # Convert position from meters to pixels and center in screen
        pixel_pos = (self.position / SCALE + np.array([WIDTH/2, HEIGHT/2])).astype(int)
        pygame.draw.circle(screen, self.color, pixel_pos, self.radius)

class NBodySimulation:
    def __init__(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("N-Body Simulation")
        self.clock = pygame.time.Clock()
        self.bodies = self.initialize_bodies()
        self.dragging = False
        self.drag_start = None
        self.current_size = BodySize.MEDIUM
        self.paused = False
        self.font = pygame.font.Font(None, 36)
        
    def initialize_bodies(self) -> List[Body]:
        # Create some example bodies (Sun and planets)
        bodies = [
            # Sun (at center)
            Body(
                mass=1.989e30,  # kg
                position=np.array([0.0, 0.0]),  # m
                velocity=np.array([0.0, 0.0]),  # m/s
                color=YELLOW,
                radius=20
            ),
            # Earth
            Body(
                mass=5.972e24,  # kg
                position=np.array([149.6e9, 0.0]),  # m (1 AU)
                velocity=np.array([0.0, 29.78e3]),  # m/s
                color=BLUE,
                radius=10
            )
        ]
        return bodies
    
    def calculate_acceleration(self, body: Body, other_bodies: List[Body]) -> np.ndarray:
        total_acceleration = np.zeros(2)
        for other in other_bodies:
            if other is not body:
                r = other.position - body.position
                r_mag = np.linalg.norm(r)
                if r_mag > 0:  # Avoid division by zero
                    total_acceleration += G * other.mass * r / (r_mag ** 3)
        return total_acceleration
    
    def screen_to_space_coords(self, screen_pos: Tuple[int, int]) -> np.ndarray:
        return (np.array(screen_pos) - np.array([WIDTH/2, HEIGHT/2])) * SCALE
    
    def handle_click_drag(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            self.dragging = True
            self.drag_start = np.array(event.pos)
        
        elif event.type == pygame.MOUSEBUTTONUP and event.button == 1 and self.dragging:
            self.dragging = False
            end_pos = np.array(event.pos)
            
            # Calculate velocity based on drag vector
            launch_vector = self.drag_start - end_pos
            velocity = launch_vector * VELOCITY_SCALE
            
            # Create new body
            new_body = Body(
                mass=self.current_size.value["mass"],
                position=self.screen_to_space_coords(self.drag_start),
                velocity=velocity,
                color=GREEN,
                radius=self.current_size.value["radius"]
            )
            self.bodies.append(new_body)
    
    def handle_input(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_1]:
            self.current_size = BodySize.SMALL
        elif keys[pygame.K_2]:
            self.current_size = BodySize.MEDIUM
        elif keys[pygame.K_3]:
            self.current_size = BodySize.LARGE
        elif keys[pygame.K_SPACE]:
            self.paused = not self.paused
    
    def draw_ui(self):
        # Draw size indicator
        size_text = f"Size: {self.current_size.name}"
        text_surface = self.font.render(size_text, True, WHITE)
        self.screen.blit(text_surface, (10, 10))
        
        # Draw pause indicator if paused
        if self.paused:
            pause_text = "PAUSED"
            text_surface = self.font.render(pause_text, True, WHITE)
            self.screen.blit(text_surface, (WIDTH - 100, 10))
        
        # Draw drag line when dragging
        if self.dragging:
            mouse_pos = pygame.mouse.get_pos()
            pygame.draw.line(self.screen, WHITE, self.drag_start, mouse_pos, 2)
    
    def update(self):
        if not self.paused:
            # Calculate and apply accelerations for all bodies
            accelerations = []
            for body in self.bodies:
                acc = self.calculate_acceleration(body, self.bodies)
                accelerations.append(acc)
                
            # Update positions
            for body, acc in zip(self.bodies, accelerations):
                body.update_position(acc, TIME_STEP)
    
    def draw(self):
        self.screen.fill(BLACK)
        for body in self.bodies:
            body.draw(self.screen)
        self.draw_ui()
        pygame.display.flip()
    
    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        running = False
                self.handle_click_drag(event)
            
            self.handle_input()
            self.update()
            self.draw()
            self.clock.tick(FPS)
        
        pygame.quit()

if __name__ == "__main__":
    simulation = NBodySimulation()
    simulation.run()